In [ ]:
from xinghe.spark import *
from app.common.json_util import *
from xinghe.s3 import *

config = {
    "spark_conf_name": "spark_4",
    "skip_success_check": True,
}

from itertools import groupby
from operator import itemgetter
from pyspark.sql.functions import from_json, to_json, struct, col
from pyspark.sql.types import StructType, StructField, StringType

DUMPS = [
    ...
]
MAX_OUTPUT_ROW_SIZE = 1024 * 1024 * 1024 * 1.5
base_input_path = f"s3a://xxx/"
already_exist_id_path = "s3://xxx/"
output_path = "s3://xxx/"
spark = new_spark_session("cc_dumps.dedup.sec", config)
sc = spark.sparkContext

# 未去重的hash data

In [ ]:
dump_paths = []
for dump in DUMPS:
    input_path = f"{base_input_path}{dump}"
    dump_paths.append(input_path)

In [ ]:
input_df = spark.read.format("json").load(dump_paths)

# 已去重的ID

In [ ]:
already_exist_id_v_df = read_any_path(spark, already_exist_id_path, config)

In [ ]:
# 定义 Schema
schema = StructType([
    StructField("hash_html", StringType(), True),
    StructField("sub_path", StringType(), True),
])
df_with_struct = already_exist_id_v_df.withColumn("json_struct", from_json(already_exist_id_v_df.value, schema))
already_exist_id_df = df_with_struct.select("json_struct.hash_html", col("json_struct.sub_path").alias("sub_path_exist"))

# filter 历史未去重的剩余id

In [ ]:
join_df = input_df.join(already_exist_id_df, on="hash_html", how="left")
undedup_id_df = join_df.filter(col("sub_path_exist").isNull()).select(["track_id", "sub_path", "hash_html"])

# 分区去重-->全局去重

In [ ]:
def deduplicate_partition(partition):
    sorted_partition = sorted(partition, key=itemgetter('hash_html'))
    return (next(group) for _, group in groupby(sorted_partition, key=itemgetter('hash_html')))
dedup_part_df = undedup_id_df.rdd.mapPartitions(deduplicate_partition).toDF(undedup_id_df.schema)

In [ ]:
dedup_df = dedup_part_df.dropDuplicates(["hash_html"])

# 写出s3

In [ ]:
struct_col = struct(dedup_df["track_id"],dedup_df["sub_path"],dedup_df["hash_html"],)
output_df = dedup_df.withColumn("value", to_json(struct_col)).select("value")

In [ ]:
config["skip_output_version"] = True
config["output_compression"] = "gz"
write_any_path(output_df, output_path, config)